In [ ]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, ZeroPadding2D, BatchNormalization, Activation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, TerminateOnNaN, TensorBoard
from tensorflow.keras import activations
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import mnist
import numpy as np

# Load CSV data
train_data = pd.read_csv(r"D:\Projects\Digit-Recognizer-ML_Flask\train.csv")

# Prepare CSV dataset
x_csv = train_data.drop('label', axis=1).values.reshape(-1, 28, 28, 1) / 255.0
y_csv = to_categorical(train_data['label'].values)

# Load MNIST dataset
(x_mnist_train, y_mnist_train), (x_mnist_test, y_mnist_test) = mnist.load_data()
x_mnist_train = x_mnist_train.reshape(-1, 28, 28, 1) / 255.0
y_mnist_train = to_categorical(y_mnist_train, 10)

x_mnist_test = x_mnist_test.reshape(-1, 28, 28, 1) / 255.0
y_mnist_test = to_categorical(y_mnist_test, 10)

# Concatenate CSV and MNIST datasets
x_combined = np.concatenate([x_csv, x_mnist_train, x_mnist_test], axis=0)
y_combined = np.concatenate([y_csv, y_mnist_train, y_mnist_test], axis=0)

# Train-validation split for combined dataset
random_seed = 2
X_train, X_val, Y_train, Y_val = train_test_split(x_combined, y_combined, test_size=0.1, random_state=random_seed)

# Define the model
model = Sequential([
    ZeroPadding2D(padding=(1, 1), input_shape=(28, 28, 1)),  # Input layer, padding added
    Conv2D(filters=32, kernel_size=(5, 5), padding='same', activation='relu'),  # Convolutional layer
    BatchNormalization(),  # Batch normalization layer
    Conv2D(filters=32, kernel_size=(5, 5), padding='same', activation='relu'),  # Convolutional layer
    BatchNormalization(),  # Batch normalization layer
    Activation(activations.relu),  # Activation layer
    MaxPooling2D(pool_size=(2, 2)),  # Max pooling layer
    ZeroPadding2D(padding=(1, 1)),  # Zero padding layer
    Dropout(0.2),  # Dropout layer
    
    Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'),  # Convolutional layer
    BatchNormalization(),  # Batch normalization layer
    Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'),  # Convolutional layer
    BatchNormalization(),  # Batch normalization layer
    Activation(activations.relu),  # Activation layer
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),  # Max pooling layer
    Dropout(0.2),  # Dropout layer
    
    Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu'),  # Convolutional layer
    BatchNormalization(),  # Batch normalization layer
    Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu'),  # Convolutional layer
    BatchNormalization(),  # Batch normalization layer
    Activation(activations.relu),  # Activation layer
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),  # Max pooling layer
    Dropout(0.3),  # Dropout layer
    
    Flatten(),  # Flatten the 3D outputs to 1D
    Dense(128, activation='relu'),  # Fully connected layer
    Dropout(0.4),  # Dropout layer
    Dense(10, activation='softmax')  # Output layer for classification
])

# Callbacks for saving the best model and learning rate reduction
checkpoint = ModelCheckpoint(
    r'best_model.keras', 
    monitor='val_accuracy',
    save_best_only=True, 
    verbose=1,
    mode='auto'
)
learning_rate_reduction = ReduceLROnPlateau(
    monitor='val_accuracy', 
    patience=3, 
    verbose=1, 
    factor=0.5, 
    min_lr=0.00001
)
early_stopping = EarlyStopping(
    monitor='val_loss',        # Моніторинг валідаційних втрат
    patience=5,                # Чекати 5 епох без покращення перед зупинкою
    restore_best_weights=True,  # Відновити ваги найкращої моделі
    mode='auto'
)
terminate_nan = TerminateOnNaN()
tensorboard = TensorBoard(
    log_dir='./public',         # Директорія для збереження логів
    histogram_freq=1,        # Записувати гістограми змінних після кожної епохи
    write_graph=True,        # Записувати граф моделі
    write_images=True,      # Записувати зображення ваг
    update_freq='epoch'     # Оновлення на основі кожної епохи
)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Image Data Generator for augmentation
datagen = ImageDataGenerator(
   rotation_range=10,          # Randomly rotate images by up to 10 degrees
   zoom_range=0.1,             # Randomly zoom images by up to 10%
   width_shift_range=0.1,      # Randomly shift images horizontally by up to 10%
   height_shift_range=0.1,     # Randomly shift images vertically by up to 10%
)

# Train the model
batch_size = 100
history = model.fit(datagen.flow(x_combined, y_combined, batch_size=batch_size),
                    epochs=10,
                    validation_data=(X_val, Y_val),
                    steps_per_epoch=x_combined.shape[0] // batch_size,
                    verbose=1,
                    callbacks=[checkpoint, learning_rate_reduction, early_stopping, terminate_nan, tensorboard],
                    shuffle=True)

# Evaluate the model
model.evaluate(X_val, Y_val)

# Plot accuracy and loss
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
model.summary()

In [10]:
from keras.layers import Input,InputLayer, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import MaxPooling2D, Dropout
from keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras
from keras import backend as K
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, TerminateOnNaN, TensorBoard
import numpy as np
from tensorflow.keras.datasets import mnist

# Load CSV data
train_data = pd.read_csv(r"D:\Projects\Digit-Recognizer-ML_Flask\train.csv")

train_data = train_data.astype('float32')

# Prepare CSV dataset
x_csv = train_data.drop('label', axis=1).values.reshape(-1, 28, 28, 1) / 255.0
y_csv = to_categorical(train_data['label'].values)

# Load MNIST dataset
(x_mnist_train, y_mnist_train), (x_mnist_test, y_mnist_test) = mnist.load_data()
x_mnist_train = x_mnist_train.reshape(-1, 28, 28, 1) / 255.0
y_mnist_train = to_categorical(y_mnist_train, 10)

x_mnist_test = x_mnist_test.reshape(-1, 28, 28, 1) / 255.0
y_mnist_test = to_categorical(y_mnist_test, 10)

# Concatenate CSV and MNIST datasets
x_combined = np.concatenate([x_csv, x_mnist_train, x_mnist_test], axis=0)
y_combined = np.concatenate([y_csv, y_mnist_train, y_mnist_test], axis=0)

# Train-validation split for combined dataset
random_seed = 42
X_train, X_val, Y_train, Y_val = train_test_split(x_combined, y_combined, test_size=0.1, random_state=random_seed)

# Building a CNN model
input_shape = (28,28,1)
X_input = Input(input_shape)

# Image Data Generator for augmentation
datagen = ImageDataGenerator(
   rotation_range=10,          # Randomly rotate images by up to 10 degrees
   zoom_range=0.1,             # Randomly zoom images by up to 10%
   width_shift_range=0.1,      # Randomly shift images horizontally by up to 10%
   height_shift_range=0.1,     # Randomly shift images vertically by up to 10%
)

# layer 1
x = Conv2D(64,(3,3),strides=(1,1),name='layer_conv1',padding='same')(X_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2,2),name='maxPool1')(x)
# layer 2
x = Conv2D(32,(3,3),strides=(1,1),name='layer_conv2',padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2,2),name='maxPool2')(x)
# layer 3
x = Conv2D(32,(3,3),strides=(1,1),name='conv3',padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2,2), name='maxPool3')(x)
# fc
x = Flatten()(x)
x = Dense(64,activation ='relu',name='fc0')(x)
x = Dropout(0.25)(x)
x = Dense(32,activation ='relu',name='fc1')(x)
x = Dropout(0.25)(x)
x = Dense(10,activation ='softmax',name='fc2')(x)

conv_model = Model(inputs=X_input, outputs=x, name='Predict')

# Callbacks for saving the best model and learning rate reduction
checkpoint = ModelCheckpoint(
    r'111.keras', 
    monitor='val_accuracy',
    save_best_only=True, 
    verbose=1,
    mode='auto'
)
learning_rate_reduction = ReduceLROnPlateau(
    monitor='val_accuracy', 
    patience=3, 
    verbose=1, 
    factor=0.5, 
    min_lr=0.00001
)
early_stopping = EarlyStopping(
    monitor='val_loss',        # Моніторинг валідаційних втрат
    patience=5,                # Чекати 5 епох без покращення перед зупинкою
    restore_best_weights=True,  # Відновити ваги найкращої моделі
    mode='auto'
)
terminate_nan = TerminateOnNaN()

# Adam optimizer
conv_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
conv_model.fit(datagen.flow(x_combined, y_combined, batch_size=100),
                    epochs=10,
                    validation_data=(X_val, Y_val),
                    steps_per_epoch=x_combined.shape[0] // 100,
                    verbose=1,
                    callbacks=[checkpoint, learning_rate_reduction, early_stopping, terminate_nan],
                    shuffle=True)
# Evaluate the model
conv_model.evaluate(X_val, Y_val)

Epoch 1/10


d:\Anaconda\envs\my-course\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1120/1120 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - accuracy: 0.7333 - loss: 0.8057
Epoch 1: val_accuracy improved from -inf to 0.98491, saving model to 111.keras
1120/1120 ━━━━━━━━━━━━━━━━━━━━ 114s 98ms/step - accuracy: 0.7334 - loss: 0.8053 - val_accuracy: 0.9849 - val_loss: 0.0487 - learning_rate: 0.0010
Epoch 2/10


d:\Anaconda\envs\my-course\lib\contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 2: val_accuracy did not improve from 0.98491
1120/1120 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9849 - val_loss: 0.0487 - learning_rate: 0.0010
Epoch 3/10
1120/1120 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - accuracy: 0.9625 - loss: 0.1359
Epoch 3: val_accuracy improved from 0.98491 to 0.98679, saving model to 111.keras
1120/1120 ━━━━━━━━━━━━━━━━━━━━ 107s 95ms/step - accuracy: 0.9625 - loss: 0.1359 - val_accuracy: 0.9868 - val_loss: 0.0436 - learning_rate: 0.0010
Epoch 4/10

Epoch 4: val_accuracy did not improve from 0.98679
1120/1120 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9868 - val_loss: 0.0436 - learning_rate: 0.0010
Epoch 5/10
1120/1120 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - accuracy: 0.9731 - loss: 0.1000
Epoch 5: val_accuracy improved from 0.98679 to 0.99116, saving model to 111.keras
1120/1120 ━━━━━━━━━━━━━━━━━━━━ 107s 95ms/step - accuracy: 0.9731 - loss: 0.1000 - val_accuracy:

[0.02721174992620945, 0.992767870426178]